In [ ]:
from sciencecrawler.pubmed import PubmedSearch, PubmedArticle
import json
from neo4jsciencecrawler import Neo4jScienceCrawler, tools as neo4jtools
import time


In [ ]:
words_covid = ["COVID-19","SARS‑CoV‑2","SARSCoV-2","COVID-2019","2019-nCoV","HCoV-19","hCoV-19","2019-nCoV","Wuhan coronavírus","Wuhan pneumonia","Pneumonia in Wuhan"]
words_hidro = ["Hydroxychloroquine","(Hydroxy)chloroquine","Chloroquine","CQ","HCQ"]

In [ ]:
USER = ""
PASSWORD = ""
URL = "bolt://localhost:7687"

In [ ]:
searchengine = PubmedSearch('covid-19 chloroquine')

In [ ]:
total = searchengine.total_search

In [ ]:
total = 50

In [ ]:
articles = searchengine.get_list_articles()

In [ ]:
articles_read = 0

In [ ]:
neo4jC = Neo4jScienceCrawler(URL,USER,PASSWORD)

def save_in_neo4j(article):
    try:
        article_json = article.to_json()
        references = article.get_references()
        neo4jC.create_article(article_json)
        neo4jC.change_metadado(article_json['id'],"is_covid",neo4jtools.check_article(article_json,words_covid))
        neo4jC.change_metadado(article_json['id'],"is_hidro",neo4jtools.check_article(article_json,words_hidro))
        for ref in references:
            ref_json = ref.to_json()
            neo4jC.create_connection_ref(article_json['id'],ref_json)
            neo4jC.change_metadado(ref_json['id'],"is_covid",neo4jtools.check_article(ref_json,words_covid))
            neo4jC.change_metadado(ref_json['id'],"is_hidro",neo4jtools.check_article(ref_json,words_hidro))
    except Exception as ex:
        print(ex)
    

In [ ]:
while articles_read < total:
    try:
        time.sleep(5)
        print(articles_read)
        searchengine.get_next_page()
        articles_current_page = searchengine.get_list_articles()
        articles_read = articles_read + len(articles_current_page)
        for article in articles_current_page:
            try:
                print(article.url)
                save_in_neo4j(article)
            except Exception as ex:
                print(ex)
    except Exception as ex:
        print(ex)
      